In [56]:
from torch.utils.data import Dataset,DataLoader
import librosa
import transformers
import tqdm
from transformers import ASTForAudioClassification,ASTFeatureExtractor
import torch
import pandas as pd
from transformers import ASTConfig
import numpy as np
import os
import shutil
from multiprocessing import Pool
import random

In [58]:
SAMPLING_RATE=22050
MAX_LENGTH=512
DURATION=5
SEGMENT=SAMPLING_RATE*DURATION

def split_num(file):
    y, _ = librosa.load(os.path.join('train_audio', file), sr=SAMPLING_RATE)
    return y.shape[0] // (SEGMENT) + 1

def random_audio_file(path='train_soundscapes'):
    files = [f for f in os.listdir(path) if f.endswith('.ogg')]
    return random.choice(files)

def tile_padding(arr):
    n=arr.shape[0]
    

'H18_20230517_001000.ogg'

In [49]:
configuration = ASTConfig()
taxonomy=pd.read_csv('taxonomy.csv')
label_nums=taxonomy.shape[0]
id2label={}
label2id={}
for i in range(label_nums):
    id2label[i]=taxonomy['primary_label'][i]
for key,value in id2label.items():
    label2id[value]=key
configuration.id2label=id2label
configuration.label2id=label2id
del taxonomy

In [48]:
df=pd.read_csv('train.csv')
df['file']=df['filename'].str.split('/').str[-1]
for i in df.itertuples():
    src_path=os.path.join('train_audio',i.filename)
    if os.path.exists(src_path):
        dst_path=os.path.join('train_audio',i.file)
        if not os.path.exists(dst_path):
            shutil.move(src_path,dst_path)
        else:continue

df=df[['primary_label','file']]
file_to_labels = df.groupby('file')['primary_label'].apply(list)
file_to_labels = file_to_labels.apply(lambda x: list(dict.fromkeys(x)))
max_labels = file_to_labels.apply(len).max()
df = pd.DataFrame(file_to_labels.tolist(), index=file_to_labels.index)
df.columns = [f'label{i}' for i in range(df.shape[1])]
df = df.reset_index()


files = df['file'].tolist()

with Pool(32) as p:
    splitnums = p.map(split_num, files)



In [51]:
df['splitnum'] = splitnums
df['accumulate']=df['splitnum'].cumsum()
df

,file,label0,label1,label2,label3,splitnum,accumulate
0,CSA03598.ogg,48124,None,None,None,11,11
1,CSA04279.ogg,50186,None,None,None,4,15
2,CSA04299.ogg,50186,None,None,None,14,29
3,CSA06193.ogg,52884,None,None,None,2,31
4,CSA06195.ogg,52884,None,None,None,21,52
...,...,...,...,...,...,...,...
28547,iNat999302.ogg,grekis,None,None,None,5,216318
28548,iNat999438.ogg,bbwduc,None,None,None,2,216320
28549,iNat999513.ogg,grekis,None,None,None,4,216324
28550,iNat999668.ogg,compau,None,None,None,3,216327


In [53]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593",
                                                  attn_implementation="sdpa",
                                                  torch_dtype=torch.bfloat16).to('cuda')
extractor=ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593",
                                              sampling_rate=SAMPLING_RATE)
model.config=configuration
extractor.sampling_rate=SAMPLING_RATE
extractor.max_length=MAX_LENGTH


In [55]:
class AstAudio(Dataset):
    def __init__(self):
        self.data=df
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        split=np.array(self.data['splitnum'])
        cum=np.array(self.data['accumulate'])
        id=cum.searchsorted(index)
        part_id=index-cum[id]+split[id]
        audio_path=os.path.join('train_audio',self.data.iloc[id].file)
        wave,_=librosa.load(audio_path,sr=SAMPLING_RATE)
        
        return part_id
b=AstAudio()
b.__getitem__(216327
              )

np.int64(3)